# Hyperparameter Search for MLP Pipeline

This notebook performs hyperparameter optimization for the MLP-based PTM prediction pipeline. It can be run for any PTM type that uses the standard schema (acet_k, gly_n, phos_y, sumo_k). It searches over architecture (hidden dimensions, dropout), learning rate, and batch size, then stores the best configuration and results under `docs/ptm_<PTM_TYPE>/`.

In [1]:
# Parameters (override with papermill: -p PTM_TYPE acet_k -p MAX_CONFIGS 150)
PTM_TYPE = "phos_y"
MAX_CONFIGS = 150

import pandas as pd
import numpy as np
import json
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    average_precision_score, matthews_corrcoef,
)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
import itertools
from datetime import datetime
warnings.filterwarnings("ignore")

DATA_DIR = Path("../datasets")
RESULTS_DIR = Path(f"../docs/ptm_{PTM_TYPE}")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
SEED = 42

In [2]:
# Parameters
PTM_TYPE = "gly_n"
MAX_CONFIGS = 150


## Load pipeline functions

We import or redefine the necessary functions from `simple_pipeline.ipynb`.

In [3]:
# Standard 20 amino acids (one-letter); index 20 = unknown
AA_ALPHABET = "ACDEFGHIKLMNPQRSTVWY"
AA_TO_IDX = {a: i for i, a in enumerate(AA_ALPHABET)}
AA_TO_IDX["X"] = 20
NUM_AAS = 21

NUMERIC_FEATURES = [
    "embedding_dispersion", "center_window_dispersion",
    "embedding_entropy", "center_window_entropy",
    "svd_entropy", "center_svd_entropy",
]

def encode_sequence(seq, window_len, aa_to_idx, num_aas):
    """One-hot encode sequence to fixed length."""
    out = np.zeros((window_len, num_aas), dtype=np.float32)
    for i, aa in enumerate(seq[:window_len]):
        idx = aa_to_idx.get(aa, 20)
        out[i, idx] = 1.0
    return out.flatten()

def build_features(df, window_len):
    seq_vecs = np.array([
        encode_sequence(s, window_len, AA_TO_IDX, NUM_AAS)
        for s in df["original_sequence"]
    ])
    num_feat = df[NUMERIC_FEATURES].values.astype(np.float32)
    return np.hstack([seq_vecs, num_feat])

class WindowDataset(Dataset):
    def __init__(self, X, y):
        # MPS (Metal) only supports float32; ensure no float64 from numpy
        self.X = torch.from_numpy(np.asarray(X, dtype=np.float32))
        self.y = torch.from_numpy(np.asarray(y, dtype=np.float32)).unsqueeze(1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=(512, 256), dropout=0.3):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.extend([nn.Linear(prev, h), nn.ReLU(), nn.Dropout(dropout)])
            prev = h
        self.backbone = nn.Sequential(*layers)
        self.head = nn.Linear(prev, 1)

    def forward(self, x):
        return self.head(self.backbone(x)).squeeze(-1)

def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auprc = average_precision_score(y_true, y_prob)
    mcc = matthews_corrcoef(y_true, y_pred)
    return {"precision": precision, "recall": recall, "f1": f1, "auprc": auprc, "mcc": mcc}

def evaluate(model, loader, device):
    model.eval()
    all_prob, all_y = [], []
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            prob = torch.sigmoid(logits).cpu().numpy()
            all_prob.append(prob)
            all_y.append(y.cpu().numpy().ravel())
    y_prob = np.concatenate(all_prob)
    y_true = np.concatenate(all_y)
    return y_true, y_prob, compute_metrics(y_true, y_prob)

## Load and prepare data

Load data for the selected PTM type and perform protein-wise train/val/test split (same as simple_pipeline.ipynb).

In [4]:
train_df = pd.read_csv(DATA_DIR / f"{PTM_TYPE}_train.csv")
test_df = pd.read_csv(DATA_DIR / f"{PTM_TYPE}_test.csv")

window_len = len(train_df["original_sequence"].iloc[0])
print(f"Window length: {window_len}")

# Protein-wise split
protein_ids = train_df["UniProt_ID"].unique()
train_proteins, val_proteins = train_test_split(
    protein_ids, test_size=0.2, random_state=SEED
)

train_split = train_df[train_df["UniProt_ID"].isin(train_proteins)].copy()
val_split = train_df[train_df["UniProt_ID"].isin(val_proteins)].copy()

# Build features
X_train = build_features(train_split, window_len)
y_train = train_split["ptm_type"].values
X_val = build_features(val_split, window_len)
y_val = val_split["ptm_type"].values
X_test = build_features(test_df, window_len)
y_test = test_df["ptm_type"].values

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

Window length: 101


Train: 11494, Val: 3064, Test: 1698


## Define hyperparameter search space

We search over:
- **Hidden dimensions**: Different architectures
- **Dropout**: Regularization strength
- **Learning rate**: Optimization speed
- **Batch size**: Training dynamics

In [5]:
# Hyperparameter grid
HYPERPARAM_GRID = {
    "hidden_dims": [
        (256, 128),
        (512, 256),
        (512, 256, 128),
        (1024, 512),
        (1024, 512, 256),
    ],
    "dropout": [0.2, 0.3, 0.4, 0.5],
    "learning_rate": [1e-4, 5e-4, 1e-3, 2e-3],
    "batch_size": [128, 256, 512],
}

# Generate all combinations, then limit to MAX_CONFIGS per run
keys = list(HYPERPARAM_GRID.keys())
values = list(HYPERPARAM_GRID.values())
full_configs = [dict(zip(keys, v)) for v in itertools.product(*values)]
random.seed(SEED)
configs = random.sample(full_configs, min(MAX_CONFIGS, len(full_configs)))

print(f"Total configurations to test: {len(configs)} (capped from {len(full_configs)})")
print(f"Sample config: {configs[0]}")

Total configurations to test: 150 (capped from 240)
Sample config: {'hidden_dims': (1024, 512), 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 256}


## Training function

Wrapper function that trains with given hyperparameters and returns results.

In [6]:
def train_with_config(
    config, X_train, y_train, X_val, y_val, X_test, y_test,
    num_epochs=15, device=None, verbose=False, seed=42
):
    """Train MLP with given hyperparameter configuration."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Set seeds
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    
    # Compute pos_weight
    n_pos = y_train.sum()
    n_neg = len(y_train) - n_pos
    pos_weight = float(n_neg) / max(n_pos, 1)
    
    # Create datasets and loaders
    train_ds = WindowDataset(X_train, y_train)
    val_ds = WindowDataset(X_val, y_val)
    train_loader = DataLoader(
        train_ds, batch_size=config["batch_size"], shuffle=True, num_workers=0
    )
    val_loader = DataLoader(
        val_ds, batch_size=config["batch_size"], shuffle=False, num_workers=0
    )
    
    # Create model
    input_dim = X_train.shape[1]
    model = MLP(
        input_dim,
        hidden_dims=config["hidden_dims"],
        dropout=config["dropout"]
    ).to(device)
    
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], dtype=torch.float32, device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    # Training loop
    best_val_auprc = 0.0
    best_state = None
    history = {"train_loss": [], "val_auprc": []}
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y.squeeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        train_loss = running_loss / len(train_loader)
        history["train_loss"].append(train_loss)
        
        # Validation
        y_val_true, y_val_prob, val_metrics = evaluate(model, val_loader, device)
        history["val_auprc"].append(val_metrics["auprc"])
        
        if val_metrics["auprc"] > best_val_auprc:
            best_val_auprc = val_metrics["auprc"]
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    
    # Load best model and evaluate on test set
    model.load_state_dict(best_state)
    test_loader = DataLoader(
        WindowDataset(X_test, y_test),
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=0
    )
    y_test_true, y_test_prob, test_metrics = evaluate(model, test_loader, device)
    
    return {
        "config": config,
        "best_val_auprc": best_val_auprc,
        "test_metrics": test_metrics,
        "history": history,
        "final_train_loss": history["train_loss"][-1]
    }

## Run hyperparameter search

Loop over all configurations and store results. This may take a while depending on the number of configs and epochs.

In [7]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
            device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

results = []
num_epochs = 15

for idx, config in enumerate(configs):
    print(f"\n[{idx+1}/{len(configs)}] Testing config:")
    print(f"  hidden_dims={config['hidden_dims']}, dropout={config['dropout']}, "
          f"lr={config['learning_rate']}, batch_size={config['batch_size']}")
    
    try:
        result = train_with_config(
            config, X_train, y_train, X_val, y_val, X_test, y_test,
            num_epochs=num_epochs, device=device, verbose=False, seed=SEED
        )
        
        results.append({
            "config_id": idx,
            "hidden_dims": str(config["hidden_dims"]),
            "dropout": config["dropout"],
            "learning_rate": config["learning_rate"],
            "batch_size": config["batch_size"],
            "best_val_auprc": result["best_val_auprc"],
            "test_precision": result["test_metrics"]["precision"],
            "test_recall": result["test_metrics"]["recall"],
            "test_f1": result["test_metrics"]["f1"],
            "test_auprc": result["test_metrics"]["auprc"],
            "test_mcc": result["test_metrics"]["mcc"],
            "final_train_loss": result["final_train_loss"]
        })
        
        print(f"  Val AUPRC: {result['best_val_auprc']:.4f}, Test AUPRC: {result['test_metrics']['auprc']:.4f}")
        
    except Exception as e:
        print(f"  ERROR: {e}")
        results.append({
            "config_id": idx,
            "hidden_dims": str(config["hidden_dims"]),
            "dropout": config["dropout"],
            "learning_rate": config["learning_rate"],
            "batch_size": config["batch_size"],
            "error": str(e)
        })

print(f"\nCompleted {len(results)} configurations.")

Using device: mps

[1/150] Testing config:
  hidden_dims=(1024, 512), dropout=0.3, lr=0.001, batch_size=256


  Val AUPRC: 0.9781, Test AUPRC: 0.9753

[2/150] Testing config:
  hidden_dims=(256, 128), dropout=0.4, lr=0.0005, batch_size=256


  Val AUPRC: 0.9728, Test AUPRC: 0.9748

[3/150] Testing config:
  hidden_dims=(256, 128), dropout=0.2, lr=0.001, batch_size=128


  Val AUPRC: 0.9763, Test AUPRC: 0.9775

[4/150] Testing config:
  hidden_dims=(1024, 512), dropout=0.5, lr=0.002, batch_size=128


  Val AUPRC: 0.9792, Test AUPRC: 0.9699

[5/150] Testing config:
  hidden_dims=(512, 256), dropout=0.3, lr=0.002, batch_size=256


...

  Val AUPRC: 0.9736, Test AUPRC: 0.9725

[147/150] Testing config:
  hidden_dims=(1024, 512), dropout=0.4, lr=0.001, batch_size=256


  Val AUPRC: 0.9769, Test AUPRC: 0.9752

[148/150] Testing config:
  hidden_dims=(512, 256, 128), dropout=0.3, lr=0.0005, batch_size=128


  Val AUPRC: 0.9798, Test AUPRC: 0.9757

[149/150] Testing config:
  hidden_dims=(1024, 512), dropout=0.3, lr=0.0005, batch_size=128


  Val AUPRC: 0.9763, Test AUPRC: 0.9753

[150/150] Testing config:
  hidden_dims=(1024, 512, 256), dropout=0.3, lr=0.001, batch_size=128


  Val AUPRC: 0.9759, Test AUPRC: 0.9685

Completed 150 configurations.


## Analyze results and find best configuration

Convert results to DataFrame, identify the best configuration by validation AUPRC, and save results.

In [8]:
results_df = pd.DataFrame(results)

# Filter out errors
valid_results = results_df[~results_df["best_val_auprc"].isna()].copy()

if len(valid_results) > 0:
    # Sort by validation AUPRC
    valid_results = valid_results.sort_values("best_val_auprc", ascending=False)
    
    print("Top 10 configurations by validation AUPRC:")
    print(valid_results[["hidden_dims", "dropout", "learning_rate", "batch_size",
                         "best_val_auprc", "test_auprc", "test_f1", "test_mcc"]].head(10).to_string())
    
    # Best configuration
    best_config = valid_results.iloc[0]
    print("\n" + "="*60)
    print("BEST CONFIGURATION:")
    print("="*60)
    print(f"Hidden dims: {best_config['hidden_dims']}")
    print(f"Dropout: {best_config['dropout']}")
    print(f"Learning rate: {best_config['learning_rate']}")
    print(f"Batch size: {best_config['batch_size']}")
    print(f"\nValidation AUPRC: {best_config['best_val_auprc']:.4f}")
    print(f"Test AUPRC: {best_config['test_auprc']:.4f}")
    print(f"Test Precision: {best_config['test_precision']:.4f}")
    print(f"Test Recall: {best_config['test_recall']:.4f}")
    print(f"Test F1: {best_config['test_f1']:.4f}")
    print(f"Test MCC: {best_config['test_mcc']:.4f}")
    print("="*60)
else:
    print("No valid results found.")

Top 10 configurations by validation AUPRC:
         hidden_dims  dropout  learning_rate  batch_size  best_val_auprc  test_auprc   test_f1  test_mcc
32   (512, 256, 128)      0.3         0.0010         128        0.982193    0.978983  0.971591  0.942109
6         (512, 256)      0.2         0.0020         128        0.981736    0.973033  0.969835  0.938421
99   (512, 256, 128)      0.2         0.0010         128        0.981491    0.969843  0.964841  0.928267
12   (512, 256, 128)      0.5         0.0010         256        0.980403    0.974643  0.970907  0.940601
147  (512, 256, 128)      0.3         0.0005         128        0.979796    0.975700  0.966111  0.930758
134  (512, 256, 128)      0.4         0.0005         256        0.979538    0.978149  0.966970  0.932475
92   (512, 256, 128)      0.5         0.0020         256        0.979455    0.969296  0.968625  0.935883
3        (1024, 512)      0.5         0.0020         128        0.979226    0.969885  0.966610  0.931876
81        (5

## Save results

Save all results to CSV and the best configuration to JSON for easy reuse.

In [9]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save all results
results_csv = RESULTS_DIR / f"hyperparameter_search_results_{timestamp}.csv"
results_df.to_csv(results_csv, index=False)
print(f"Saved all results to: {results_csv}")

# Save best configuration
if len(valid_results) > 0:
    best_config_dict = {
        "timestamp": timestamp,
        "hidden_dims": eval(best_config["hidden_dims"]),  # Convert string back to tuple
        "dropout": float(best_config["dropout"]),
        "learning_rate": float(best_config["learning_rate"]),
        "batch_size": int(best_config["batch_size"]),
        "metrics": {
            "best_val_auprc": float(best_config["best_val_auprc"]),
            "test_precision": float(best_config["test_precision"]),
            "test_recall": float(best_config["test_recall"]),
            "test_f1": float(best_config["test_f1"]),
            "test_auprc": float(best_config["test_auprc"]),
            "test_mcc": float(best_config["test_mcc"]),
        }
    }
    
    best_config_json = RESULTS_DIR / f"best_mlp_config_{timestamp}.json"
    with open(best_config_json, "w") as f:
        json.dump(best_config_dict, f, indent=2)
    print(f"Saved best configuration to: {best_config_json}")
    
    # Also save a human-readable summary
    summary_txt = RESULTS_DIR / f"hyperparameter_search_summary_{timestamp}.txt"
    with open(summary_txt, "w") as f:
        f.write("Hyperparameter Search Summary\n")
        f.write("="*60 + "\n\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Total configurations tested: {len(configs)}\n")
        f.write(f"Valid results: {len(valid_results)}\n\n")
        f.write("Best Configuration:\n")
        f.write(f"  Hidden dims: {best_config_dict['hidden_dims']}\n")
        f.write(f"  Dropout: {best_config_dict['dropout']}\n")
        f.write(f"  Learning rate: {best_config_dict['learning_rate']}\n")
        f.write(f"  Batch size: {best_config_dict['batch_size']}\n\n")
        f.write("Best Metrics:\n")
        for k, v in best_config_dict["metrics"].items():
            f.write(f"  {k}: {v:.4f}\n")
    print(f"Saved summary to: {summary_txt}")
    
    # Save a summary figure (top configs by val AUPRC)
    import matplotlib.pyplot as plt
    top = valid_results.head(10)
    fig, ax = plt.subplots(figsize=(10, 4))
    x = range(len(top))
    ax.bar([i - 0.2 for i in x], top["best_val_auprc"], width=0.2, label="Val AUPRC")
    ax.bar([i for i in x], top["test_auprc"], width=0.2, label="Test AUPRC")
    ax.bar([i + 0.2 for i in x], top["test_mcc"], width=0.2, label="Test MCC")
    ax.set_xticks(x)
    ax.set_xticklabels([f"{r['hidden_dims']}" for _, r in top.iterrows()], rotation=45, ha="right")
    ax.set_ylabel("Score")
    ax.legend()
    ax.set_title(f"Top 10 configs by val AUPRC (PTM: {PTM_TYPE})")
    plt.tight_layout()
    fig_path = RESULTS_DIR / f"top10_configs_{timestamp}.png"
    plt.savefig(fig_path)
    plt.close()
    print(f"Saved summary figure to: {fig_path}")

Saved all results to: ../docs/ptm_phos_y/hyperparameter_search_results_20260208_015008.csv
Saved best configuration to: ../docs/ptm_phos_y/best_mlp_config_20260208_015008.json
Saved summary to: ../docs/ptm_phos_y/hyperparameter_search_summary_20260208_015008.txt


Saved summary figure to: ../docs/ptm_phos_y/top10_configs_20260208_015008.png
